In [9]:
import pandas as pd
import missingno
import ast
import sys
from zipfile import ZipFile

In [2]:
zip_name = "lastfm-dataset-1K.zip"
with ZipFile(zip_name, 'r') as z:
    folder = "lastfm-dataset-1K/"
    
    #Read song csv
    song_file = folder +"userid-timestamp-artid-artname-traid-traname.tsv"
    songs = pd.read_csv(z.open(song_file), sep='\t', 
                        error_bad_lines=False, 
                        header=None, parse_dates=[1], 
                        dtype={0:'category', 2:'category',3:'category', 
                          4:'category',5:'category'})
    songs.columns = ['user_id', 'timestamp', 'artist_id', 
                     'artist_name', 'track_id', 'track_name']
    
    #Read user csv
    user_file = folder +"userid-profile.tsv"
    column_types = {'#id': 'category', 'gender':'category', 
                    'age':'float32', 'country':'category'}
    users = pd.read_csv(z.open(user_file), sep='\t', parse_dates=[4], dtype=column_types, error_bad_lines=False) \
                .rename(columns={'#id': 'user_id'})
    

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


In [3]:
zip_name = "song_tags_combined.zip"
with ZipFile(zip_name, 'r') as z:
    folder = "song_tags_combined/"
    
    file = folder +"song_tags.csv"
    song_tags = pd.read_csv(z.open(file), error_bad_lines=False)

In [4]:
song_tags.head()

,artist_name,track_name,tags
0,Boy Division,Love Will Tear Us Apart,"[{'count': 100, 'name': 'rock', 'url': 'https:..."
1,Death Cab For Cutie,Soul Meets Body,"[{'count': 100, 'name': 'indie', 'url': 'https..."
2,The Knife,Heartbeats,"[{'count': 100, 'name': 'electronic', 'url': '..."
3,Muse,Starlight,"[{'count': 100, 'name': 'alternative rock', 'u..."
4,The Killers,When You Were Young,"[{'count': 100, 'name': 'rock', 'url': 'https:..."


# Top N Tags, All tags in one column

In [17]:
all = False
n = sys.maxsize if all else 3

for index, row in song_tags[0:10].iterrows():
    tags = row['tags']
    str_tag = str(tags)
    if(str_tag[0] == '['):
        x = ast.literal_eval(str(tags).lower())
        tag_list = []
        for i in range(min(n, len(x))):
            if(song_tags.loc[index, 'artist_name'].lower() != x[i]['name']):
                tag_list.append(x[i]['name'])
        song_tags.loc[index, 'tag_names'] = ", ".join(str(x) for x in tag_list)
    else:
        song_tags.loc[index, 'tag_names'] = str_tag

song_tags

,artist_name,track_name,tags,tag_names
0,Boy Division,Love Will Tear Us Apart,"[{'count': 100, 'name': 'rock', 'url': 'https:...","rock, covers, cool covers"
1,Death Cab For Cutie,Soul Meets Body,"[{'count': 100, 'name': 'indie', 'url': 'https...","indie, indie rock, alternative"
2,The Knife,Heartbeats,"[{'count': 100, 'name': 'electronic', 'url': '...","electronic, synthpop, swedish"
3,Muse,Starlight,"[{'count': 100, 'name': 'alternative rock', 'u...","alternative rock, rock, alternative"
4,The Killers,When You Were Young,"[{'count': 100, 'name': 'rock', 'url': 'https:...","rock, indie rock, indie"
...,...,...,...,...
1498674,The Back Horn,The Black Hole Birthday,[],NaN
1498675,Dio - Distraught Overlord,Lord'S Prayer,"[{'count': 100, 'name': 'visual kei', 'url': '...",NaN
1498676,Penicillin,Monster,[],NaN
1498677,Blood,#01 (Bonus Track),[],NaN


# Top N Tags, Column per tag

In [8]:
all = False
n = sys.maxsize if all else 10

for index, row in song_tags.iterrows():
    tags = row['tags']
    str_tag = str(tags)
    if(str_tag[0] == '['):
        x = ast.literal_eval(str(tags).lower())
        for i in range(min(n, len(x))):
            song_tags.loc[index, 'tag_' + str(i)] = x[i]['name']
    else:
        song_tags.loc[index, 'tag_names'] = str_tag

song_tags

/Users/manon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/manon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,artist_name,track_name,tags,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
0,Boy Division,Love Will Tear Us Apart,"[{'count': 100, 'name': 'rock', 'url': 'https:...",rock,covers,cool covers,joy division,NaN,NaN,NaN,NaN,NaN,NaN
1,Death Cab For Cutie,Soul Meets Body,"[{'count': 100, 'name': 'indie', 'url': 'https...",indie,indie rock,alternative,death cab for cutie,rock,mellow,favorites,indie pop,emo,alternative rock
2,The Knife,Heartbeats,"[{'count': 100, 'name': 'electronic', 'url': '...",electronic,synthpop,swedish,electro,electronica,the knife,female vocalists,indie,electropop,synth pop
3,Muse,Starlight,"[{'count': 100, 'name': 'alternative rock', 'u...",alternative rock,rock,alternative,muse,british,indie,progressive rock,indie rock,beautiful,britpop
4,The Killers,When You Were Young,"[{'count': 100, 'name': 'rock', 'url': 'https:...",rock,indie rock,indie,alternative,alternative rock,the killers,american,killers,favorites,awesome
5,Kanye West,Love Lockdown,"[{'count': 100, 'name': 'Hip-Hop', 'url': 'htt...",hip-hop,kanye west,hip hop,pop,catchy,rnb,love lockdown,electronic,rap,love
6,Muse,Time Is Running Out,"[{'count': 100, 'name': 'alternative rock', 'u...",alternative rock,rock,alternative,muse,indie,british,progressive rock,indie rock,seen live,britpop
7,Bloc Party,Banquet,"[{'count': 100, 'name': 'indie', 'url': 'https...",indie,indie rock,alternative,rock,bloc party,british,britpop,alternative rock,post-punk,00s
8,Radiohead,All I Need,"[{'count': 100, 'name': 'alternative', 'url': ...",alternative,alternative rock,radiohead,rock,british,indie,beautiful,electronic,experimental,love
9,Snow Patrol,Chasing Cars,"[{'count': 100, 'name': 'indie', 'url': 'https...",indie,alternative,rock,indie rock,love,snow patrol,alternative rock,britpop,mellow,greys anatomy


# Get tags set 

In [13]:
all = False
n = sys.maxsize if all else 10

tag_set = set()



for index, row in song_tags.iterrows():
    tags = row['tags']
    str_tag = str(tags)
    if(str_tag[0] == '['):
        x = ast.literal_eval(str(tags))
        for i in range(min(len(x), n)):
            tag_set.add(x[i]['name'])


tag_set

{'spooky songs',
 'canciones tesoro',
 'dance-punk',
 'The Damage Manual I wish I had',
 'love vent',
 'Kicks ass',
 'pee-why',
 'incredibly cool and different',
 'Great Psyche',
 'game boy',
 'Hitchhikers Guide to the Galaxy',
 'road tripping',
 'In a Nutshell',
 'lay ya ass down',
 'jesse james',
 'Favorit songs',
 'For the club',
 'motion',
 'j4zz r0ck',
 'Benny Benassi',
 'Los Toreros',
 'ice mc',
 'piedpp tagged',
 'Dancefloor Mayhem',
 'genres: rock and roll 50s - 60s',
 'Awesome solo',
 'frenchrap',
 'Rachael Starr Till there was you',
 'Deutsch Gut',
 'cendres ascendantes',
 'pale',
 'cool trip',
 'awnnnnnnnn those days are so gone',
 'somekinda all-ages party type deal',
 'highly syncopated drum beats',
 'feel it',
 'proto hip-hop',
 'florida songs',
 'Better off Dead',
 'acoustic dreamy',
 'sidereal space',
 'best of 2002',
 'rapider than horsepower',
 'baroque rock',
 'Electric Eel Shock',
 'indie instrumental chill out',
 'Daniel Merriweather',
 'teqqen',
 'Moszeed unter St

In [14]:
fname = "all_tags.txt" if all else "top_tags.txt"

with open(fname, "w") as output:
    output.write(str(tag_set))

# Merge song tags with listening history

In [22]:
song_tags.drop(columns=['tags'], inplace=True)
song_tags.rename(columns={"tag_names": "tags"})
song_tags.head()

KeyError: "['tags'] not found in axis"

In [86]:
new_df = pd.merge(songs, song_tags, on=['artist_name','track_name'])

In [88]:
new_df

,user_id,timestamp,artist_id,artist_name,track_id,track_name,tags,tag_names
0,user_000001,2009-05-04 23:08:57+00:00,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,"[{'count': 100, 'name': 'dance', 'url': 'https...","dance, House"
1,user_000001,2009-05-04 13:54:10+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15),[],
2,user_000001,2009-04-25 14:09:18+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15),[],
3,user_000001,2009-04-21 18:23:34+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15),[],
4,user_000001,2009-04-20 13:27:17+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15),[],
...,...,...,...,...,...,...,...,...
19098790,user_001000,2008-02-03 03:17:41+00:00,e549424e-8a71-4257-acf7-18e798127cae,Katrah-Quey,7d570229-2ffb-47d1-8e15-789ba702c1bd,"France Horns, Scanning, And Boombap",[],
19098791,user_001000,2008-02-03 03:15:50+00:00,e549424e-8a71-4257-acf7-18e798127cae,Katrah-Quey,649599d2-cbdc-4e02-81b9-1d382274a02a,12 Years Of Readyness,"[{'count': 100, 'name': 'instrumental', 'url':...","instrumental, instrumental hip-hop, beats, jaz..."
19098792,user_001000,2008-02-03 03:14:42+00:00,e549424e-8a71-4257-acf7-18e798127cae,Katrah-Quey,cc92a82d-7661-412a-857e-4ce2fc29c71b,Neglect,"[{'count': 100, 'name': 'instrumental', 'url':...","instrumental, instrumental hip-hop, heard on l..."
19098793,user_001000,2008-02-03 03:08:50+00:00,e549424e-8a71-4257-acf7-18e798127cae,Katrah-Quey,57daae1d-d8fe-4dba-a447-cfa81cb41882,An Orange Splash On Things,"[{'count': 100, 'name': 'instrumental hip-hop'...","instrumental hip-hop, heard on last-fm"
